# Gai/Gen: Text-to-Text Generation using Gai with OpenAI

## Setting Up

1. Make a copy of `.env.example` in this directory and rename it to `.env`. Fill in the value for `OPENAI_API_KEY` with your OpenAI API key.

    ```bash
    OPENAI_API_KEY=your-api-key
    ```

2. Create a conda environment called `TTT`, if not already created, and install the dependencies:

    ```bash
    sudo apt update -y && sudo apt install ffmpeg git git-lfs -y
    conda create -n TTT python=3.10.10 -y
    conda activate TTT
    cd ../../gai-gen
    pip install -e ".[TTT]"
    ```
---

## Chat Completion


Using OpenAI API

In [1]:
print("> OpenAI API")

from dotenv import load_dotenv
load_dotenv()
import os
if not os.environ.get("OPENAI_API_KEY"):
    raise Exception(
        "OPENAI_API_KEY not found in environment variables")
from openai import OpenAI
client = OpenAI()
client.api_key = os.environ["OPENAI_API_KEY"]

# Streaming
print("STREAMING")
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role":"user","content":"Tell me a one paragraph story"}],
    stream=True,
    max_tokens=1000,
)
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end="",flush=True)

# No Streaming
print("\nNO STREAMING")
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role":"user","content":"Tell me a one paragraph story"}],
    max_tokens=1000,
)
print(response.choices[0].message.content)

> OpenAI API
STREAMING
Once upon a time, in the lush valley of Veridis, lived Carina, a humble young maiden who had a passion for discovering forgotten herbs and healing the ill. One day, she stumbled upon a dying dragon, encrusted with scales reflecting the hues of the rainbow under the sunlight. Overcoming her profound fear, she gathered her courage, tended to the dragon with her medicinal skills. The dragon, in return, granted Carina an enchanted emerald pendant that bestowed her with an everlasting ability to heal any ailment. Thus, Carina became a beacon of hope for the inhabitants of Veridis, healing illnesses, mending hearts, and forever remembered as the savior of the Rainbow Dragon.
NO STREAMING
Once upon a time in the heart of a densely populated city, a homeless man notorious for his hunched back and cheerful smile, named Tom, discovered a winning lottery ticket in the trash bin. He didn't know the value of the ticket, but to his surprise, when he cashed it in, it turned out

Using GAI API

In [5]:
# 1. Text-to-Text Generation
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')

# Streaming
print("STREAMING")
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}],stream=True)
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end='',flush=True)

# No Streaming
print("\nNO STREAMING")
response = gen.create(messages=[{'role':'USER','content':'Tell me a one paragraph short story.'},{'role':'ASSISTANT','content':''}], max_tokens=100,stream=False)
print(response.choices[0].message.content)

2024-06-03 15:49:53 INFO gai.gen.Gaigen:Gaigen: Loading generator gpt-4...
2024-06-03 15:49:53 INFO gai.gen.ttt.TTT:Using engine OpenAI_TTT...
2024-06-03 15:49:53 DEBUG gai.gen.ttt.OpenAI_TTT:OpenAI_TTT.create: model_params={'max_tokens': 100, 'temperature': 0.7, 'top_p': 1, 'presence_penalty': 0.0, 'frequency_penalty': 0.0, 'stop': None, 'logit_bias': {}, 'n': 1}


STREAMING
Once upon a time, in a small town nestled between hills and a sparkling river, lived a humble old painter named Milo. Milo was known for his unique ability to bring portraits to life, not literally, but through his vivid details and colors that seemed to dance off the canvas. One day, a mysterious stranger arrived, requesting a portrait of his lost love. As Milo worked his magic, he discovered the woman in the portrait resembled his own lost love. In the end, the stranger turned out to

2024-06-03 15:49:58 DEBUG gai.gen.ttt.OpenAI_TTT:OpenAI_TTT.create: model_params={'max_tokens': 100, 'temperature': 0.7, 'top_p': 1, 'presence_penalty': 0.0, 'frequency_penalty': 0.0, 'stop': None, 'logit_bias': {}, 'n': 1}



NO STREAMING
In the heart of a bustling city, there was an old, forgotten bookstore, run by a kind, elderly man named Mr. Hawthorne. One day, a young girl named Lucy, disheartened by the digital world, stumbled upon this hidden gem. Enchanted by the musty smell of old books and the soft crackling sound of flipping pages, she spent countless hours there, losing herself in the world of words. Over time, the bond between the girl and the old man strengthened


---

## Function Calling

Using OpenAI API

In [1]:
import os,json
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
if not os.environ.get("OPENAI_API_KEY"):
    raise Exception(
        "OPENAI_API_KEY not found in environment variables")
openai.api_key = os.environ["OPENAI_API_KEY"]
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role":"user","content":"Who is the current president of Singapore?"}],
    max_tokens=100,
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }                   
    ],
)
from gai.common.notebook import highlight
highlight("ChatCompletion:")
print(response)
highlight("function:")
print("function=",response.choices[0].message.tool_calls[0].function.name)
highlight("arguments:")
print("arguments=",response.choices[0].message.tool_calls[0].function.arguments)


# tool = {}
# tool["arguments"]=""
# for chunk in response:
#     if chunk.choices[0].delta.tool_calls and chunk.choices[0].delta.tool_calls[0].function.name:
#         tool["name"] = chunk.choices[0].delta.tool_calls[0].function.name
#     elif chunk.choices[0].delta.tool_calls and chunk.choices[0].delta.tool_calls[0].function.arguments:
#         tool["arguments"] += chunk.choices[0].delta.tool_calls[0].function.arguments
# print(json.dumps(tool, indent=4)+"\n")



ChatCompletion(id='chatcmpl-9XfHjseWxijSn63KammRkQk8Lj5f5', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9E3aNoGM5cEfbTWxCDxWbkJA', function=Function(arguments='{\n  "search_query": "current president of Singapore"\n}', name='google'), type='function')]))], created=1717810551, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=230, total_tokens=248))


function= google


arguments= {
  "search_query": "current president of Singapore"
}


Using Gai API

In [18]:
from gai.gen import Gaigen
gen = Gaigen.GetInstance().load('gpt-4')

response = gen.create(
    messages="user: Who is the current president of Singapore?\nassistant:",
    max_tokens=100,
    tools=[
        {
            "type": "function",
            "function": {
                "name": "google",
                "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search_query": {
                            "type": "string",
                            "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                        }
                    },
                    "required": ["search_query"]
                }
            }
        }                   
    ],
    stream=False)
print("function=",response.choices[0].message.tool_calls[0].function.name)
print("arguments=",response.choices[0].message.tool_calls[0].function.arguments)

2024-06-03 16:00:46 DEBUG gai.gen.Gaigen:Gaigen.load: Generator is already loaded. Skip loading.
2024-06-03 16:00:46 DEBUG gai.gen.ttt.OpenAI_TTT:OpenAI_TTT.create: model_params={'max_tokens': 100, 'temperature': 0.7, 'top_p': 1, 'presence_penalty': 0.0, 'frequency_penalty': 0.0, 'stop': None, 'logit_bias': {}, 'n': 1, 'tools': [{'type': 'function', 'function': {'name': 'google', 'description': "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or w

function= google
arguments= {
  "search_query": "current president of Singapore"
}


---

## Gai API Service



1. Build the docker image for the Gai API service.

In [25]:
!docker build --build-arg CATEGORY=ttt -f ../../gai-gen/Dockerfile.TTT -t gai-ttt:latest ../../gai-gen    

[+] Building 0.0s (0/0)  docker:default
[+] Building 0.0s (1/1)                                          docker:default
 => [internal] load build definition from Dockerfile.TTT                   0.0s
[+] Building 0.2s (1/3)                                          docker:default
 => [internal] load build definition from Dockerfile.TTT                   0.0s
 => => transferring dockerfile: 1.28kB                                     0.0s
 => [internal] load metadata for docker.io/kakkoii1337/cu118-cp310-base:1  0.1s
 => [auth] kakkoii1337/cu118-cp310-base:pull token for registry-1.docker.  0.0s
[+] Building 0.3s (2/3)                                          docker:default
 => [internal] load build definition from Dockerfile.TTT                   0.0s
 => => transferring dockerfile: 1.28kB                                     0.0s
 => [internal] load metadata for docker.io/kakkoii1337/cu118-cp310-base:1  0.3s
 => [auth] kakkoii1337/cu118-cp310-base:pull token for registry-1.docker.  0.0s


Start API service

In [26]:
!docker container rm -f gai-ttt
!docker run -d \
            -e DEFAULT_GENERATOR=gpt-4 \
            -e OPENAI_API_KEY=${OPENAI_API_KEY} \
            --gpus all \
            -p 12031:12031 \
            --name gai-ttt \
            gai-ttt:latest

gai-ttt
a8a18008d7251d0beb55ba01416f0a131a0a313d6498c768e830ccd5a4f5e787


In [27]:
%%bash
echo STREAMING
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d '{"model":"gpt-4", 
        "messages": [ 
            {"role": "user","content": "Tell me a story"}, 
            {"role": "assistant","content": ""} 
        ], "stream":true, "max_tokens":1000 }' | python ../../gai-gen/tests/integration_tests/print_delta.py


STREAMING
Once upon a time, in a serene village named Sunnyville, there lived an old, kind-hearted man named Henry. Everyone in Sunnyville adored Henry for his charming personality and his heart of gold. Despite being aged, Henry was renowned for his extraordinary gardening skills. His garden was the heart of Sunnyville, filled with vibrant flowers, lush green trees, and rare exotic plants.

One day, Henry discovered a peculiar seed while gardening. It was unlike any seed he had ever seen before. Intrigued, he decided to plant it in the middle of his garden. He watered it, gave it plenty of sunlight and tended to it patiently, hoping for a beautiful plant.

Days turned into weeks, and weeks into months, but there was no sign of a sprout. The village folk started to tease Henry, calling the unusual seed a dud. However, Henry’s faith didn’t waver. He believed in the seed and continued to nurture it with all his love and care.

A year passed by, and on a cool spring morning, something mag